In [223]:
import numpy as np

class Polynom:
    def __init__(self, arg):
        if isinstance(arg, int):
            assert arg >= 0, f"Expected non-negative degree, got {arg}."
            self.deg = arg
            self.coeffs = np.zeros(arg + 1)
        elif isinstance(arg, np.ndarray):
            assert arg.ndim == 1, "Coefficients must be a 1D array."
            self.coeffs = arg.astype(float)
            self.deg = len(self.coeffs) - 1
        else:
            raise TypeError("Polynom constructor expects an int or a 1D numpy array.")

    def __call__(self, x: float):
        result = 0.0
        power = 1.0
        for c in self.coeffs:
            result += c * power
            power *= x
        return result

    def __str__(self):
        return f"<Polynom deg={self.deg}, coeffs={self.coeffs}>"

    def set_coeff(self, id_coeff:int, coeff:float):
        assert id_coeff<=self.deg and id_coeff>=0, ValueError(
            f"Expected a coefficient the polynom has, got {id_coeff}.")
        self.coeffs[id_coeff] = coeff
    
    def get_coeff(self, id_coeff:int):
        assert id_coeff<=self.deg and id_coeff>=0, ValueError(
            f"Expected a coefficient the polynom has, got {id_coeff}.")
        return self.coeffs[id_coeff]

    def set_deg(self, deg:int):
        assert deg>=0, ValueError(
            f"Expected a positive polynom degree, got {deg}.")
        if deg < self.deg:
            self.coeffs = self.coeffs[: deg]
        else:
            self.coeffs = np.append(self.coeffs, np.zeros(deg - self.deg))
        self.deg = deg
    
    def get_deg(self):
        return self.deg

    def derive(self):
        if self.deg == 0:
            self.coeffs = np.array([0.0])
        else:
            deriv = np.empty(self.deg)
            for i in range(1, self.deg+1):
                deriv[i - 1] = i * self.coeffs[i]

            self.deg -= 1
            self.coeffs = deriv
    
    def add(self, p: np.ndarray):
        n = max(self.deg, p.get_deg())
        res = np.zeros(n+1)
        for i in range(self.deg + 1):
            res[i] += self.coeffs[i]
        for i in range(p.get_deg() + 1):
            res[i] += p.coeffs[i]

        last_non_zero = 0
        for i in reversed(range(n + 1)):
            if abs(res[i]) > 1e-14:
                last_non_zero = i
                break
        
        self.deg = last_non_zero
        self.coeffs = res[:last_non_zero + 1]

    def mul_scalar(self, scalar: float):
        self.coeffs *= scalar
        if scalar == 0:
            self.deg = 0

    def mul_X(self):
        res = np.zeros(self.deg + 2)
        for i in range(self.deg + 1):
            res[i + 1] = self.coeffs[i]

        self.deg += 1
        self.coeffs = res

    def copy(self):
        p = Polynom(self.deg)
        p.coeffs = np.copy(self.coeffs)
        return p

def next_assoc_legendre(l:int, m:int, P0:Polynom, P1:Polynom):
    P = Polynom(P1.get_deg()+1)

    a = P1.copy()
    b = P0.copy()

    a.mul_scalar((2*l + 1) / (l - m + 1))
    a.mul_X()

    b.mul_scalar(-(l + m) / (l - m + 1))

    P.add(a)
    P.add(b)
    return P

class AssociatedLegendrePolynomsCalculator:
    """Class for associated Legendre Polynoms for m=2 (weak lensing).
    """
    def __init__(self):
        self.P2 = Polynom(np.array([3, 0, -3]))
        self.P3 = Polynom(np.array([0, 15, 0, -15]))
        self.polynoms = [self.P2, self.P3]
    
    def __str__(self):
        return f"<AssociatedLegendrePolynoms n={len(self.polynoms)}>"
    
    def __call__(self, nb_l:int):
        assert nb_l>=0, ValueError(
            f"Expected l>=0, got {nb_l}.")

        if nb_l<=2:
            return self.polynoms[:l]
        
        P0 = self.P2.copy()
        P1 = self.P3.copy()
        for l in range(3, nb_l+1):
            P_next = next_assoc_legendre(l, 2, P0, P1)
            P0, P1 = P1, P_next
            self.polynoms.append(P_next)
        return self.polynoms


In [228]:
from astropy.io import fits

file_path = 'DES-Y3_xipm_and_KiDS-1000_COSEBIs_2.0_300.0.fits'

with fits.open(file_path) as hdul:
    # hdul.info()
    xip_data = hdul['xip'].data
    xim_data = hdul['xim'].data
    desy3_covmat = hdul['COVMAT'].data

xi_minus_data = xim_data['VALUE']
xi_minus_ang = xim_data['ANG']
xi_plus_data = xip_data['VALUE']
xi_plus_ang = xip_data['ANG']

polynoms = AssociatedLegendrePolynomsCalculator()

nb_l=5
ps = polynoms(nb_l=nb_l)
for i in range(nb_l):
    print(f"P(l={i}, m=2) : {ps[i]}")

P(l=0, m=2) : <Polynom deg=2, coeffs=[ 3.  0. -3.]>
P(l=1, m=2) : <Polynom deg=3, coeffs=[  0.  15.   0. -15.]>
P(l=2, m=2) : <Polynom deg=4, coeffs=[ -7.5   0.   60.    0.  -52.5]>
P(l=3, m=2) : <Polynom deg=5, coeffs=[   0.   -52.5    0.   210.     0.  -157.5]>
P(l=4, m=2) : <Polynom deg=6, coeffs=[  13.125    0.    -249.375    0.     669.375    0.    -433.125]>


In [6]:
def Gp(x:float, l:int, m:int=2):
    return - ( (l-m**2) / (1-x**2) + 0.5 * l * (l-1) ) * ps[l](x) + (l+m) * (x / (1-x**2)) * ps[l-1](x)

def Gm(theta:float):
    return m * ( (l-1) * x / (1-x**2) * ps[l](x) - (l+m) * (1 / (1-x**2)) * ps[l-1](x))

def ClEE(l:int, i:int, j:int):
    return 0.0

def ClBB(l:int, i:int, j:int):
    return 0.0

def xip(theta:float, i:int, j:int):
    xi = 0.0
    for l in range(nb_l):
        xi += (2*l + 1)/(2*np.pi*(l+1)**2) * (Gp(np.cos(theta)) + Gm(np.cos(theta))) * (ClEE(l, i, j) + ClBB(l, i, j))
    return xi
